# Read the log file

In [8]:
# Read data file
import peforth, re
import numpy as np
%f s" 5.log" readTextFile constant data-file // ( -- text ) entire data file 
# 1.log 是自由落體 

reDef data-file


In [9]:
# 從整個檔案切開每行，只挑要的
ss = [s for s in peforth.execute('data-file').pop().split('\n')] 
x = [s for s in ss if ("X=" in s)]
y = [s for s in ss if ("Fuel=" in s)]
c = [s for s in ss if ("command" in s)]

%f x :> [:4] --> # x 看前幾行
%f y :> [:4] --> # y 看前幾行
%f c :> [:4] --> # c 看前幾行
    
%f x py> len(pop()) --> # 看有幾行
%f y py> len(pop()) --> # 看有幾行
%f c py> len(pop()) --> # 看有幾行，最後一個 command 之前就撞毀了，所以少了一個。


x :> [:4] --> ['X=2500m, Y=2500m, HSpeed=0m/s VSpeed=0m/s', 'X=2500m, Y=2498m, HSpeed=0m/s VSpeed=-4m/s', 'X=2500m, Y=2493m, HSpeed=0m/s VSpeed=-7m/s', 'X=2500m, Y=2484m, HSpeed=0m/s VSpeed=-10m/s'] (<class 'list'>)
x 看前幾行

y :> [:4] --> ['Fuel=500l, Angle=0°, Power=0 (0.0m/s2)', 'Fuel=500l, Angle=0°, Power=0 (0.0m/s2)', 'Fuel=500l, Angle=0°, Power=0 (0.0m/s2)', 'Fuel=499l, Angle=0°, Power=1 (1.0m/s2)'] (<class 'list'>)
y 看前幾行

c :> [:4] --> ['command 0 0', 'command 0 0', 'command 0 1', 'command 0 0'] (<class 'list'>)
c 看前幾行

x py> len(pop()) --> 64 (<class 'int'>)
看有幾行

y py> len(pop()) --> 64 (<class 'int'>)
看有幾行

c py> len(pop()) --> 63 (<class 'int'>)
看有幾行，最後一個 command 之前就撞毀了，所以少了一個。



In [10]:
xarray = []
for i in range(len(x)):
    s = x[i]
    m = re.match(r"X=(?P<X>\d+)\D*(?P<Y>\d+)\D*?(?P<hspeed>-?\d+)\D*?(?P<vspeed>-?\d+).*", s)
    xarray += m.groups()
xarray = list(map(int,xarray))
%f xarray :> [:16] -->
%f xarray count nip -->    


xarray :> [:16] --> [2500, 2500, 0, 0, 2500, 2498, 0, -4, 2500, 2493, 0, -7, 2500, 2484, 0, -10] (<class 'list'>)
xarray count nip --> 256 (<class 'int'>)


In [11]:
yarray = []
for i in range(len(y)):
    s = y[i]
    m = re.match(r"Fuel=(?P<fuel>\d+)\D*?(?P<angle>-?\d+)\D*(?P<power>\d+).*", s)
    yarray += m.groups()
yarray = list(map(int,yarray))
%f yarray :> [:16] -->
%f yarray count nip -->    

yarray :> [:16] --> [500, 0, 0, 500, 0, 0, 500, 0, 0, 499, 0, 1, 499, 0, 0, 498] (<class 'list'>)
yarray count nip --> 192 (<class 'int'>)


In [12]:
carray = []
for i in range(len(c)):
    s = c[i]
    m = re.match(r"\D*?(?P<rotate>\d+)\s+(?P<power>\d+)", s)
    carray += m.groups()
carray = list(map(int,carray))+[99,99]  # 最後一個補上撞毀的那次 command 讓矩陣完整
%f carray :> [:16] -->
%f carray count nip -->    

carray :> [:16] --> [0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 1] (<class 'list'>)
carray count nip --> 128 (<class 'int'>)


In [13]:
xarray = np.array(xarray).reshape([-1,4])
yarray = np.array(yarray).reshape([-1,3])
carray = np.array(carray).reshape([-1,2])

In [14]:
%f xarray count nip -->
%f yarray count nip -->
%f carray count nip -->

xarray count nip --> 64 (<class 'int'>)
yarray count nip --> 64 (<class 'int'>)
carray count nip --> 64 (<class 'int'>)


### 以下是從 log 檔讀出來的數據 

In [15]:
a = []
for i in range(len(xarray)):
    a.append(list(xarray[i]) + list(yarray[i]) + list(carray[i]))
%f a count nip --> # X Y HSpeed VSpeed Fuel Angle Power Rotate Power 後兩個是下一步的 command 
[ (i,a[i]) for i in range(len(a))]
    

a count nip --> 64 (<class 'int'>)
X Y HSpeed VSpeed Fuel Angle Power Rotate Power 後兩個是下一步的 command



[(0, [2500, 2500, 0, 0, 500, 0, 0, 0, 0]),
 (1, [2500, 2498, 0, -4, 500, 0, 0, 0, 0]),
 (2, [2500, 2493, 0, -7, 500, 0, 0, 0, 1]),
 (3, [2500, 2484, 0, -10, 499, 0, 1, 0, 0]),
 (4, [2500, 2472, 0, -14, 499, 0, 0, 0, 1]),
 (5, [2500, 2457, 0, -17, 498, 0, 1, 0, 0]),
 (6, [2500, 2438, 0, -20, 498, 0, 0, 0, 0]),
 (7, [2500, 2416, 0, -24, 498, 0, 0, 0, 1]),
 (8, [2500, 2391, 0, -27, 497, 0, 1, 0, 0]),
 (9, [2500, 2362, 0, -30, 497, 0, 0, 0, 1]),
 (10, [2500, 2330, 0, -33, 496, 0, 1, 0, 0]),
 (11, [2500, 2295, 0, -37, 496, 0, 0, 0, 1]),
 (12, [2500, 2257, 0, -40, 495, 0, 1, 0, 2]),
 (13, [2500, 2217, 0, -41, 493, 0, 2, 0, 3]),
 (14, [2500, 2175, 0, -42, 490, 0, 3, 0, 4]),
 (15, [2500, 2134, 0, -42, 486, 0, 4, 0, 4]),
 (16, [2500, 2092, 0, -41, 482, 0, 4, 0, 3]),
 (17, [2500, 2050, 0, -42, 479, 0, 3, 0, 4]),
 (18, [2500, 2008, 0, -42, 475, 0, 4, 0, 3]),
 (19, [2500, 1966, 0, -43, 472, 0, 3, 0, 4]),
 (20, [2500, 1924, 0, -42, 468, 0, 4, 0, 3]),
 (21, [2500, 1881, 0, -43, 465, 0, 3, 0, 4]),
 (